In [391]:
import pandas as pd
import math
import numpy as np
from collections import Counter
import datetime
import pandasql as ps

In [392]:
test_set = pd.read_excel('C:/DATASET_TEST/test_data_2022.xlsx')
test_set = test_set.set_index('row_id')
test_set.isna().mean().sort_values(ascending = False)

icd_code                 0.239746
insured_id               0.000000
date_of_birth            0.000000
gender                   0.000000
employer_id              0.000000
medical_provider_code    0.000000
date_of_service          0.000000
service_name             0.000000
service_quantity         0.000000
service_amount           0.000000
dtype: float64

In [393]:
#-вычисление возраста страховщика на момент обращения
test_set['AGE'] = ((test_set['date_of_service']-test_set['date_of_birth']).dt.days)/365.25 

#test_set.head(25)
#-вычисление возраста страховщика на момент обращения
test_set['AGE'] = ((test_set['date_of_service']-test_set['date_of_birth']).dt.days)/365.25 

#test_set.head(25)
test_set["g_a_category"] = 'na' #-столбец с категориями клиентов
#-функция присваивает категорию клиенту, в зависимости от возраста и пола
def cat_age_gender(row):
    if row['gender'] == 'М' and row['AGE'] >= 0 and row['AGE'] < 18:
        return 'Дети-М'
    if row['gender'] == 'Ж' and row['AGE'] >= 0 and row['AGE'] < 18:
        return 'Дети-Ж'
    if row['gender'] == 'М' and row['AGE'] >= 18 and row['AGE'] < 60:
        return 'Взрослые-М'
    if row['gender'] == 'Ж' and row['AGE'] >= 18 and row['AGE'] < 60:
        return 'Взрослые-Ж'
    if row['gender'] == 'М' and row['AGE'] >= 60:
        return 'Пожилые-М'
    if row['gender'] == 'Ж' and row['AGE'] >= 60:
        return 'Пожилые-Ж'
    return row['g_a_category']
test_set["g_a_category"] = test_set.apply(cat_age_gender, axis=1) #применение функции ко всем строкам датафрейма
len(test_set)

23479

In [394]:
#сортировка данных по id клиента, коду мед.учреждения и дате обращения (для удобства работы)
test_set = test_set.sort_values(by=['insured_id','medical_provider_code','date_of_service'])

In [395]:
# добавляю столбец для уникальных номеров для каждого "CASE"
test_set['cases']=1

In [396]:
# для удобства, индексирую уже отсортированный df
test_set = test_set.reset_index()

In [397]:
# копирую номера index в отдельный столбец 'ind'
test_set['ind'] = test_set.index


In [398]:
#исходя из определения case, в столбец case, для каждой строки заносится номер case, к которому она относится
for index in test_set['ind']:
    if index == 0:
        pass
    elif (test_set.loc[index,'date_of_service'] - test_set.loc[index-1,'date_of_service']).days <=14 and test_set.loc[index,'medical_provider_code']==test_set.loc[index-1,'medical_provider_code'] and test_set.loc[index,'insured_id']==test_set.loc[index-1,'insured_id']:
        test_set.loc[index,'cases'] = test_set.loc[index-1,'cases']
    else:
        test_set.loc[index,'cases'] = test_set.loc[index-1,'cases']+1

In [399]:
#столбец с уникальным номером визита, к которому относится строка
test_set['visit_id'] = 1

In [400]:
#для каждой строки проставляется номер визита, к которому она относится
for index in test_set['ind']:
    if index == 0:
        pass
    elif test_set.loc[index,'date_of_service'] == test_set.loc[index-1,'date_of_service'] and test_set.loc[index,'medical_provider_code'] == test_set.loc[index-1,'medical_provider_code'] and test_set.loc[index,'insured_id'] == test_set.loc[index-1,'insured_id']:
        test_set.loc[index,'visit_id'] = test_set.loc[index-1,'visit_id']
    else:
        test_set.loc[index,'visit_id'] = test_set.loc[index-1,'visit_id']+1

In [401]:
test_set['case_diag'] = 'n/a'

In [402]:
test_set['diag_group'] = 'n/a'

In [403]:
test_set['case_length'] = 0


In [404]:
def case_len(row):
    dd = list(test_set[test_set['cases']==row['cases']]['date_of_service'])
    ff =((dd[-1] - dd[0])).days
    return ff + 1

In [405]:
test_set['case_length'] = test_set.apply(case_len, axis=1)

In [406]:
###test_set['visits_per_case'] = 0

In [407]:
###def visits_per_case(row):
###    return len(set(test_set[test_set['cases']==row['cases']]['visit_id']))

In [408]:
###test_set['visits_per_case'] = test_set.apply(visits_per_case, axis=1)

In [409]:
###def price_per_case(row):
###    return sum(test_set[test_set['cases']==row['cases']]['service_amount'])

In [410]:
###test_set['price_per_case'] = test_set.apply(price_per_case, axis=1)

In [411]:
###def price_per_visit(row):
###    return sum(test_set[test_set['visit_id']==row['visit_id']]['service_amount'])

In [412]:
###test_set['price_per_visit'] = test_set.apply(price_per_visit, axis=1)

In [413]:
def top_client(row):
    return sum(test_set[test_set['insured_id']==row['insured_id']]['service_amount'])
test_set['overall_client_amount'] = test_set.apply(top_client, axis=1)

In [414]:
test_set2 = test_set.dropna(subset = ['icd_code']).copy() #DF для работы с диагнозами, без NaN значений
   

In [415]:
def case_diag(row):
    lst = list(test_set2[test_set2['cases']==row['cases']]['icd_code'])
    counts = Counter(lst)
    return (sorted(lst, key=lambda x: -counts[x]))[0]
    
test_set2['case_diag'] = test_set2.apply(case_diag, axis=1)

In [416]:
#функция, заполняющая столбец группы заболеваний по первому символу диагноза кейса
def diag_group(row):
    return (row['case_diag'])[0]

In [417]:
test_set2['diag_group'] = test_set2.apply(diag_group,axis=1)



Примечание*

Данные в столбце service_amount я считал, как данные за весь набор услуг.
Так как в описании не указано, что это стоимость одной услуги
- service_amount - сумма выплат (стоимость оказанных услуг в рублях)

**Что нужно посчитать:**

1. Количество медицинских кейсов
2. Количество визитов в клинику
3. Средняя длительность медицинского кейса в днях
4. Среднее количество визитов на один кейс
5. Средняя сумма выплат на один кейс
6. Средняя сумма выплат на один визит

a) в целом по выборке

In [418]:
ps.sqldf("select count(distinct cases) as 'Количество медицинских кейсов', count(distinct visit_id) as 'Количество визитов в клинику',\
        AVG(case_length) as 'Средняя длительность медицинского кейса в днях', \
        (1.0*count(distinct visit_id)/count(distinct cases)) as 'Среднее количество визитов на один кейс', (SUM(overall_client_amount)/count(distinct cases)) as 'Средняя сумма выплат на один кейс',\
        (SUM(overall_client_amount)/count(distinct visit_id)) as 'Средняя сумма выплат на один кейс' \
        from test_set")


,Количество медицинских кейсов,Количество визитов в клинику,Средняя длительность медицинского кейса в днях,Среднее количество визитов на один кейс,Средняя сумма выплат на один кейс,Средняя сумма выплат на один кейс
0,10624,15890,5.864091,1.49567,8976.329338,6001.543291


b) в разрезе по ТОП-10 страхователям

In [419]:
ps.sqldf("select insured_id, count(distinct cases) as 'Количество медицинских кейсов', count(distinct visit_id) as 'Количество визитов в клинику',\
        AVG(case_length) as 'Средняя длительность медицинского кейса в днях', \
        (1.0*count(distinct visit_id)/count(distinct cases)) as 'Среднее количество визитов на один кейс', (overall_client_amount/count(distinct cases)) as 'Средняя сумма выплат на один кейс',\
        (overall_client_amount/count(distinct visit_id)) as 'Средняя сумма выплат на один кейс', \
        overall_client_amount as 'Сумма выплат на страховщика'  from test_set GROUP BY insured_id,overall_client_amount\
        ORDER BY overall_client_amount desc limit 10")

,insured_id,Количество медицинских кейсов,Количество визитов в клинику,Средняя длительность медицинского кейса в днях,Среднее количество визитов на один кейс,Средняя сумма выплат на один кейс,Средняя сумма выплат на один кейс,Сумма выплат на страховщика
0,994512603003,2,3,3.857143,1.5,48584.00,32389.333333,97168.0
1,13537741703,2,3,8.000000,1.5,42026.25,28017.500000,84052.5
2,8359512125,1,1,1.000000,1.0,55000.00,55000.000000,55000.0
3,1310406510903,1,1,1.000000,1.0,43000.00,43000.000000,43000.0
4,923129684103,1,2,4.000000,2.0,42820.00,21410.000000,42820.0
5,342860445303,1,4,13.000000,4.0,42012.00,10503.000000,42012.0
6,749475875803,1,7,12.000000,7.0,40410.00,5772.857143,40410.0
7,63993766403,1,3,18.000000,3.0,33016.00,11005.333333,33016.0
8,839514923,2,3,5.166667,1.5,16300.00,10866.666667,32600.0
9,1243731435303,1,1,1.000000,1.0,32340.00,32340.000000,32340.0


c) в разрезе по укрупненным группам по полу и возрасту (Дети - М, Дети - Ж, Взрослые - М, Взрослые - Ж, Пожилые - М, Пожилые - Ж)
Дети - от 0 до 17 лет , взрослые - от 18 до 59 лет , пожилые - после 60 лет

In [420]:
ps.sqldf("select g_a_category as 'Укрупненная группа по полу и возрасту', count(distinct cases) as 'Количество медицинских кейсов', count(distinct visit_id) as 'Количество визитов в клинику',\
        AVG(case_length) as 'Средняя длительность медицинского кейса в днях', \
        (1.0*count(distinct visit_id)/count(distinct cases)) as 'Среднее количество визитов на один кейс', (SUM(overall_client_amount)/count(distinct cases)) as 'Средняя сумма выплат на один кейс',\
        (SUM(overall_client_amount)/count(distinct visit_id)) as 'Средняя сумма выплат на один кейс' \
        from test_set GROUP BY g_a_category\
        ORDER BY g_a_category")

,Укрупненная группа по полу и возрасту,Количество медицинских кейсов,Количество визитов в клинику,Средняя длительность медицинского кейса в днях,Среднее количество визитов на один кейс,Средняя сумма выплат на один кейс,Средняя сумма выплат на один кейс
0,Взрослые-Ж,5767,8659,5.923616,1.501474,9106.699362,6065.173256
1,Взрослые-М,3053,4608,5.931009,1.509335,8886.170056,5887.473346
2,Дети-Ж,619,867,5.406040,1.400646,8612.727625,6149.110035
3,Дети-М,635,850,4.277778,1.338583,5351.917795,3998.197412
4,Пожилые-Ж,343,571,7.254945,1.664723,12584.518950,7559.527145
5,Пожилые-М,207,335,5.459566,1.618357,12900.841981,7971.565045


d) В разрезе по группам заболеваний на основе показателя “Диагноз кейса”. 
При вычислении показателя диагноз кейса предусмотреть чистку некорректных данных.

In [423]:
ps.sqldf("SELECT diag_group as 'Диагноз кейса', count(distinct cases) as 'Количество медицинских кейсов', count(distinct visit_id) as 'Количество визитов в клинику',\
        AVG(case_length) as 'Средняя длительность медицинского кейса в днях', \
        (1.0*count(distinct visit_id)/count(distinct cases)) as 'Среднее количество визитов на один кейс', (SUM(overall_client_amount)/count(distinct cases)) as 'Средняя сумма выплат на один кейс',\
        (SUM(overall_client_amount)/count(distinct visit_id)) as 'Средняя сумма выплат на один кейс' \
        from test_set2 GROUP BY diag_group HAVING diag_group >= '0' ORDER BY case_diag DESC")

,Диагноз кейса,Количество медицинских кейсов,Количество визитов в клинику,Средняя длительность медицинского кейса в днях,Среднее количество визитов на один кейс,Средняя сумма выплат на один кейс,Средняя сумма выплат на один кейс
0,Ш,3,10,17.800000,3.333333,13333.333333,4000.000000
1,Х,3,3,1.000000,1.000000,12943.333333,12943.333333
2,Ф,1,1,1.000000,1.000000,800.000000,800.000000
3,У,1,1,1.000000,1.000000,800.000000,800.000000
4,Т,9,15,11.166667,1.666667,12534.166667,7520.500000
5,Р,1,1,1.000000,1.000000,900.000000,900.000000
6,П,12,17,3.434783,1.416667,5530.333333,3903.764706
7,О,3,11,17.666667,3.666667,34716.666667,9468.181818
8,Н,12,13,2.538462,1.083333,7873.833333,7268.153846
9,М,51,101,9.575000,1.980392,11254.043137,5682.734653
